## Get Tweets
In this notebook, we get Donald Trump tweets from the Twitter API using twython and store them locally in MongoDB.

In [2]:
# grab some sample tweets using twython before getting full 3200 dump

from twython import Twython # pip install twython
import time # standard lib

CONSUMER_KEY = '9ZU8k8vFaFTHAq1qRnP8KzqD9'
CONSUMER_SECRET = 'Yk8qC2aaknPzub0MxMuSXlgeNUr0ZhKhtkx8ESTvcygeq8M66T'
ACCESS_KEY = '762006412968984576-EuysowzBy7uYI6Y3yQyfReVOyQPFOtw'
ACCESS_SECRET = 'wUISkVgmLletYDfTckJjwMHf2jnk96uI6P18lSjA3oSpt'

twitter = Twython(CONSUMER_KEY,CONSUMER_SECRET,ACCESS_KEY,ACCESS_SECRET)

# single tweet
sample = twitter.get_user_timeline(screen_name="realDonaldTrump",count=1)

# multiple tweets
multiple_samples = twitter.get_user_timeline(screen_name="realDonaldTrump",count=5)

print multiple_samples


[{u'contributors': None, u'truncated': False, u'text': u'"@LindaHarden: @realDonaldTrump America loves Trump and @mike_pence -- praying for you every day. Stay strong. #TrumpPence2016 #NeverHillary', u'is_quote_status': False, u'in_reply_to_status_id': None, u'id': 769740626497679360, u'favorite_count': 2142, u'source': u'<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', u'retweeted': False, u'coordinates': None, u'entities': {u'symbols': [], u'user_mentions': [{u'id': 532513408, u'indices': [1, 13], u'id_str': u'532513408', u'screen_name': u'LindaHarden', u'name': u'Linda Harden'}, {u'id': 25073877, u'indices': [15, 31], u'id_str': u'25073877', u'screen_name': u'realDonaldTrump', u'name': u'Donald J. Trump'}, {u'id': 22203756, u'indices': [56, 67], u'id_str': u'22203756', u'screen_name': u'mike_pence', u'name': u'Mike Pence'}], u'hashtags': [{u'indices': [111, 126], u'text': u'TrumpPence2016'}, {u'indices': [127, 140], u'text': u'NeverHillary'}], u

In [3]:
# create json file of tweets

import json
with open('datatest.txt', 'w') as outfile:
    json.dump(multiple_samples, outfile)

In [4]:
# read formatted sample to get a feel for data
# grab "id": of first (= most recent) tweet, later use as starting id for larger data pull

with open('datatest.txt', 'r') as f:
    line = f.readline() # read only the first tweet/line
    tweet = json.loads(line) # load it as Python dict
    print(json.dumps(tweet, indent=4)) # pretty-print

[
    {
        "contributors": null, 
        "truncated": false, 
        "text": "\"@LindaHarden: @realDonaldTrump America loves Trump and @mike_pence -- praying for you every day. Stay strong. #TrumpPence2016 #NeverHillary", 
        "is_quote_status": false, 
        "in_reply_to_status_id": null, 
        "id": 769740626497679360, 
        "favorite_count": 2142, 
        "source": "<a href=\"http://twitter.com/download/android\" rel=\"nofollow\">Twitter for Android</a>", 
        "retweeted": false, 
        "coordinates": null, 
        "entities": {
            "symbols": [], 
            "user_mentions": [
                {
                    "indices": [
                        1, 
                        13
                    ], 
                    "screen_name": "LindaHarden", 
                    "id": 532513408, 
                    "name": "Linda Harden", 
                    "id_str": "532513408"
                }, 
                {
                    "indices": [

In [15]:
# pull tweets, this time 16 loops of (maximum) 200 tweets at a time with a timer between calls
# to avoid hitting the rate limit and getting blocked

CONSUMER_KEY = '9ZU8k8vFaFTHAq1qRnP8KzqD9'
CONSUMER_SECRET = 'Yk8qC2aaknPzub0MxMuSXlgeNUr0ZhKhtkx8ESTvcygeq8M66T'
ACCESS_KEY = '762006412968984576-EuysowzBy7uYI6Y3yQyfReVOyQPFOtw'
ACCESS_SECRET = 'wUISkVgmLletYDfTckJjwMHf2jnk96uI6P18lSjA3oSpt'

twitter = Twython(CONSUMER_KEY,CONSUMER_SECRET,ACCESS_KEY,ACCESS_SECRET)
tweets = []

lis = [769740626497679360] ## this is the latest starting tweet id
for i in range(0, 16): 
    user_timeline = twitter.get_user_timeline(screen_name="realDonaldTrump",
    count=200, max_id=lis[-1])
    time.sleep(200) ## rest between api calls to avoid rate limit
    
    for tweet in user_timeline:
        tweets.append(tweet) 
        #print tweet['text'] ## optional print data to confirm it's working
        lis.append(tweet['id']) ## append tweet id's so next iter knows where to begin
                

In [16]:
# confirm tweets stored

len(tweets)

3199

In [17]:
# send tweets to mongo

from pymongo import MongoClient

client = MongoClient()
db = client.test

def into_mongodb(collection, results):
    coll = collection
    for tweet in tweets:
        coll.insert_one(tweet)
        
into_mongodb(db.trumptweets3, tweets)

In [13]:
# check data stored in mongo

cursor = db.trumptweets.find()
for document in cursor[1:2]:
    print(document)

{u'contributors': None, u'truncated': False, u'text': u'Thank you Windham, New Hampshire! #TrumpPence16 #MAGA https://t.co/ZL4Q01Q49s', u'is_quote_status': False, u'in_reply_to_status_id': None, u'id': 762110918721310721L, u'favorite_count': 15984, u'source': u'<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', u'retweeted': False, u'coordinates': None, u'entities': {u'symbols': [], u'user_mentions': [], u'hashtags': [{u'indices': [34, 47], u'text': u'TrumpPence16'}, {u'indices': [48, 53], u'text': u'MAGA'}], u'urls': [], u'media': [{u'expanded_url': u'http://twitter.com/realDonaldTrump/status/762110918721310721/photo/1', u'sizes': {u'large': {u'h': 1008, u'w': 2048, u'resize': u'fit'}, u'small': {u'h': 335, u'w': 680, u'resize': u'fit'}, u'medium': {u'h': 591, u'w': 1200, u'resize': u'fit'}, u'thumb': {u'h': 150, u'w': 150, u'resize': u'crop'}}, u'url': u'https://t.co/ZL4Q01Q49s', u'media_url_https': u'https://pbs.twimg.com/media/CpOP0smXYAA93ZY.jpg',